# Notebook 5.5 - Testing the feature "Income"

# Import libraries

In [51]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy import stats
from scipy.special import inv_boxcox

# Choose the city

In [52]:
# Choose the city ('Madrid', 'Barcelona', 'Valencia', or 'Combined')
city = 'Combined'

# Load the cleaned data set incl. the feature "income"

In [53]:
#Ensure that column ZIP_CODE is being read as a string
dtype_dict = {
    'ZIP_CODE': str
}

In [54]:
if city.lower() == 'madrid':
    data = pd.read_csv('../../data/5_cleaned_and_feature_engineering/feature_income/madrid_cleaned_incl_income.csv', dtype=dtype_dict)
elif city.lower() == 'barcelona':
    data = pd.read_csv('../../data/5_cleaned_and_feature_engineering/feature_income/barcelona_cleaned_incl_income.csv', dtype=dtype_dict)
elif city.lower() == 'valencia':
    data = pd.read_csv('../../data/5_cleaned_and_feature_engineering/feature_income/valencia_cleaned_incl_income.csv', dtype=dtype_dict)
elif city.lower() == 'combined':
    #Read all 3 datasets and add a column to indicate source dataset
    madrid_df = pd.read_csv('../../data/5_cleaned_and_feature_engineering/feature_income/madrid_cleaned_incl_income.csv', dtype=dtype_dict)
    valencia_df = pd.read_csv('../../data/5_cleaned_and_feature_engineering/feature_income/valencia_cleaned_incl_income.csv', dtype=dtype_dict)
    barcelona_df = pd.read_csv('../../data/5_cleaned_and_feature_engineering/feature_income/barcelona_cleaned_incl_income.csv', dtype=dtype_dict)
    madrid_df['Madrid'] = True
    madrid_df['Valencia'] = False
    madrid_df['Barcelona'] = False    
    valencia_df['Madrid'] = False
    valencia_df['Valencia'] = True
    valencia_df['Barcelona'] = False    
    barcelona_df['Madrid'] = False
    barcelona_df['Valencia'] = False
    barcelona_df['Barcelona'] = True
    data = pd.concat([madrid_df, valencia_df, barcelona_df], ignore_index=True)

In [55]:
data.head()

,ASSETID,PRICE,CONSTRUCTEDAREA,ROOMNUMBER,BATHNUMBER,AMENITYID,HASPARKINGSPACE,PARKINGSPACEPRICE,HASTERRACE,HASLIFT,...,AVERAGE_DISPOSABLE_INCOME,LABOR_INCOME,EXEMPT_INCOME,GROSS_INCOME,SOCIAL_SECURITY_CONTRIBUTIONS,SELF_ASSESSMENT_QUOTA,DISPOSABLE_INCOME,Madrid,Valencia,Barcelona
0,A10000037964896093228,255000,97,3,2,3,0,1.0,0,1,...,33099.0,1.597692e+09,76274309.0,2.045548e+09,62934177.0,396256316.0,1.586413e+09,True,False,False
1,A10000072440601830803,82000,62,2,1,3,0,1.0,0,1,...,20458.0,6.965424e+08,36917201.0,8.145684e+08,35486524.0,99378073.0,6.797218e+08,True,False,False
2,A10000538600815177437,133000,67,3,1,3,0,1.0,1,0,...,20458.0,6.965424e+08,36917201.0,8.145684e+08,35486524.0,99378073.0,6.797218e+08,True,False,False
3,A10000654405436195291,204000,180,3,2,3,0,1.0,0,1,...,22618.0,1.311609e+09,54700472.0,1.530129e+09,61816758.0,208884677.0,1.259502e+09,True,False,False
4,A10000872160480475600,161000,54,2,1,3,0,1.0,0,0,...,22399.0,1.097802e+09,48959836.0,1.301282e+09,51743635.0,178718832.0,1.070854e+09,True,False,False


# Defining X and y

__Transform target variable PRICE as optimized in notebook 5.1__

In [56]:
# Define the lambda value
specified_lambda = 0

# Apply Box-Cox transformation using the specified lambda value
price_transformed = stats.boxcox(data['PRICE'], lmbda=specified_lambda)

# Add the transformed PRICE back to the DataFrame
data['PRICE_TRANSFORMED'] = price_transformed

__Assign features as X and target as Y__

In [57]:
X = data.drop(columns=['PRICE', 'PRICE_TRANSFORMED'])
y = data['PRICE_TRANSFORMED']

# Drop features not relevant for modelling

In [58]:
#Drop ASSETID as it is only an identifier, drop ZIP_CODE as the base model only includes base features
X = X.drop(columns=['ASSETID', 'ZIP_CODE'])

In [59]:
#Drop all columns with income data that are not on a per capita basis
X = X.drop(columns=['NUMER_OF_DECLARATIONS', 'LABOR_INCOME', 'EXEMPT_INCOME', 'GROSS_INCOME', 'SOCIAL_SECURITY_CONTRIBUTIONS', 'SELF_ASSESSMENT_QUOTA', 'DISPOSABLE_INCOME'])

# Splitting the data into training and test set

In [60]:
# Split the dataset into training and testing set with 7:3 ratio
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Check the shapes of the resulting datasets
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (104827, 45)
X_test shape: (44927, 45)
y_train shape: (104827,)
y_test shape: (44927,)


# Target encoding of base feature NEIGHBORHOOD

In [61]:
# Calculate average LOG_PRICE for each NEIGHBORHOOD only with training set
neighborhood_means = y_train.groupby(X_train['NEIGHBORHOOD']).mean()

# Assign average means to both training and test set
X_train['NEIGHBORHOOD_ENCODED'] = X_train['NEIGHBORHOOD'].map(neighborhood_means)
X_test['NEIGHBORHOOD_ENCODED'] = X_test['NEIGHBORHOOD'].map(neighborhood_means)

# Drop the NEIGHBORHOOD column from both training and testing sets
X_train = X_train.drop(columns=['NEIGHBORHOOD'])
X_test = X_test.drop(columns=['NEIGHBORHOOD'])

# Inspect data before running the model

In [62]:
X_test.head()

,CONSTRUCTEDAREA,ROOMNUMBER,BATHNUMBER,AMENITYID,HASPARKINGSPACE,PARKINGSPACEPRICE,HASTERRACE,HASLIFT,HASAIRCONDITIONING,HASNORTHORIENTATION,...,PERIOD_201803,PERIOD_201806,PERIOD_201809,PERIOD_201812,AVERAGE_GROSS_INCOME,AVERAGE_DISPOSABLE_INCOME,Madrid,Valencia,Barcelona,NEIGHBORHOOD_ENCODED
55930,49,1,1,3,0,1.0,0,1,0,0,...,0,0,0,1,31338.0,25181.0,True,False,False,12.975745
110070,74,4,1,3,0,1.0,0,1,1,0,...,0,1,0,0,399730.0,310380.0,False,False,True,12.503598
133225,113,4,2,3,0,1.0,1,1,1,0,...,0,1,0,0,399730.0,310380.0,False,False,True,12.569330
57088,42,2,1,3,0,1.0,0,0,0,0,...,0,0,0,1,35015.0,27842.0,True,False,False,11.841607
103352,60,3,1,3,0,1.0,0,1,0,1,...,0,0,0,1,399730.0,310380.0,False,False,True,12.112270


# Test the model with new feature

__Define the best parameters found by GridSearchCV in Notebook 5.1__

In [63]:
best_params = {
    'colsample_bytree': 0.8,
    'learning_rate': 0.1,
    'max_depth': 10,
    'n_estimators': 500,
    'subsample': 0.9
}

__Running the optimized model new feature__

In [64]:
# Initialize XGBoost model with the best parameters
xgb_model = xgb.XGBRegressor(**best_params)

# Train the model on the Box-Cox transformed target
xgb_model.fit(X_train, y_train)

# Predictions in Box-Cox transformed scale
y_pred_boxcox_xgb = xgb_model.predict(X_test)

# Transform predictions back to the original scale
y_pred_xgb = inv_boxcox(y_pred_boxcox_xgb, specified_lambda)
y_test_price = inv_boxcox(y_test, specified_lambda)

# Evaluation metrics on the original scale
mae_xgb = mean_absolute_error(y_test_price, y_pred_xgb)
mse_xgb = mean_squared_error(y_test_price, y_pred_xgb)
rmse_xgb = np.sqrt(mse_xgb)
mape_xgb = mean_absolute_percentage_error(y_test_price, y_pred_xgb)

# Evaluation metrics on the Box-Cox transformed scale
mae_boxcox_xgb = mean_absolute_error(y_test, y_pred_boxcox_xgb)
mse_boxcox_xgb = mean_squared_error(y_test, y_pred_boxcox_xgb)
rmse_boxcox_xgb = np.sqrt(mse_boxcox_xgb)
mape_boxcox_xgb = mean_absolute_percentage_error(y_test, y_pred_boxcox_xgb)

# Print the metrics
print("Metrics on the original PRICE scale:")
print(f"MAE: {mae_xgb:.4f}")
print(f"MSE: {mse_xgb:.4f}")
print(f"RMSE: {rmse_xgb:.4f}")
print(f"MAPE: {mape_xgb:.4f}")

print("\nMetrics on the Box-Cox transformed scale:")
print(f"MAE: {mae_boxcox_xgb:.4f}")
print(f"MSE: {mse_boxcox_xgb:.4f}")
print(f"RMSE: {rmse_boxcox_xgb:.4f}")
print(f"MAPE: {mape_boxcox_xgb:.4f}")

Metrics on the original PRICE scale:
MAE: 44213.7425
MSE: 10290799552.3887
RMSE: 101443.5782
MAPE: 0.1288

Metrics on the Box-Cox transformed scale:
MAE: 0.1244
MSE: 0.0318
RMSE: 0.1784
MAPE: 0.0100
